### Dataset G (from Jonathan's Code), q=1

In [2]:
# Import our functions
from moment_conversion import *
from ssid import *
from simulate import *

# Import other things that might be useful
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.linalg import subspace_angles
import scipy.io as sio
import pickle
import sys
sys.path.insert(0, '..')

# System orders
q = 1  # dimension of the data
p = 2  # dimension of the state space
k = p  # Hankel parameter
m = 3  # dimension of the inputs

# Simulated data size
N = 256000
    
seeds = [27642, 75802, 67482, 64738, 11526, 78492, 52745]
for sdx, seed in enumerate(seeds):
    print(sdx)
    np.random.seed(seed)
    
    # Set system parameters
    theta = np.pi/400
    A = np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
    B = np.random.normal(0,1,size=(p,m)) * 0.01 # generate_input_matrix(p,m) * .1
    C = np.random.normal(0,1,size=(q,p)) * 1e4
    D = np.random.normal(0,1,size=(q,m)) * 0.20 #generate_input_matrix(q,m) * .1
    Q = np.eye(p) * .001
    d = np.zeros(q)

    # Set prior parameters
    x0 = np.zeros(p)
    Q0 = np.eye(p) * 0.001

    # Generate inputs
    muu = np.ones(m)
    Qu = np.eye(m)
    u = np.random.multivariate_normal(muu,Qu,size=N)

    ### Run simulation ###

    # Get initial diag z for unitizing
    y, x, u, z, _ = simulate_driven_bernoulli_lds(N,x0,Q0,A,B,Q,C,D,d,muu,Qu,inputs=u)

    z_reshaped = future_past_Hankel_order_stream(z, k, q, flip=True)
    sig_z = np.cov(z_reshaped)[: q, q : 2*q]
    diag_z = np.diag(sig_z)

    # Resimulate with unitizing data
    y, x, u, z, C_new = simulate_driven_bernoulli_lds(N,x0,Q0,A,B,Q,C,D,d,muu,Qu,diag_z=diag_z,inputs=u)
    
    # moment conversion
    y_reshaped = future_past_Hankel_order_stream(y, k, q).T
    u_reshaped = future_past_Hankel_order_stream(u, k, m).T
    mu_zs, mu_us, sigma_zz, sigma_uu, sigma_zu = fit_mu_sigma_bernoulli_driven(y_reshaped, u_reshaped)

    # rearrange sigma, get estimate of covariance w 
    sigma_zz_full = tril_to_full(sigma_zz, 2 * k * q)
    sigma_what = get_sigmaw_driven(sigma_uu, sigma_zz_full, sigma_zu)

    # cholesky decompose R
    R = get_R(sigma_what)

    # run n4sid
    Ahat,Bhat,Chat,Dhat,_,_,_,ss = driven_n4sid(R,k,m,p,q)

    # Store results
    np.savez('data/em-inits/datasetG_q1/bestLDS-bigC-%d.npz' % sdx, seed=seed, A=A, B=B, C=C, C_new=C_new, D=D, 
                Q=Q, Q0=Q0, x0=x0, muu=muu, Qu=Qu, y=y, z=z, u=u, x=x, Ahat=Ahat, Bhat=Bhat, Chat=Chat, Dhat=Dhat)
    data_dict = {'seed':seed, 'A':A, 'B':B, 'C':C, 'C_new':C_new, 'D':D, 'Q':Q, 'Q0':Q0, 'x0':x0, 'muu':muu, 
                 'Qu':Qu, 'y':y, 'z':z, 'u':u, 'x':x, 'Ahat':Ahat, 'Bhat':Bhat, 'Chat':Chat, 'Dhat':Dhat}
    sio.savemat('data/em-inits/datasetG_q1/bestLDS-bigC-%d.mat' % sdx, data_dict)

0
1
2
3
4
5
6


## Dataset D, q=1

In [4]:
# Import our functions
from moment_conversion import *
from ssid import *
from simulate import *

# Import other things that might be useful
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.linalg import subspace_angles
import scipy.io as sio
import pickle
import sys
sys.path.insert(0, '..')

# System orders
q = 1  # dimension of the data
p = 2  # dimension of the state space
k = p  # Hankel parameter
m = 3  # dimension of the inputs

# Simulated data size
N = 256000
    
seeds = [27642, 75802, 67482, 64738, 11526, 78492, 52745]
for sdx, seed in enumerate(seeds):
    print(sdx)
    np.random.seed(seed)
    
    # Set system parameters
    theta = np.pi/48
    A = np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
    B = generate_input_matrix(p,m) * .1
    C = np.random.normal(0,1,size=(q,p)) * 1e4
    D = generate_input_matrix(q,m) * .1
    Q = np.eye(p) * .0001
    d = np.zeros(q)

    # Set prior parameters
    x0 = np.zeros(p)
    Q0 = np.eye(p) * 0.0001

    # Generate inputs
    muu = np.ones(m)
    Qu = np.eye(m)
    u = np.random.multivariate_normal(muu,Qu,size=N)

    ### Run simulation ###

    # Get initial diag z for unitizing
    y, x, u, z, _ = simulate_driven_bernoulli_lds(N,x0,Q0,A,B,Q,C,D,d,muu,Qu,inputs=u)

    z_reshaped = future_past_Hankel_order_stream(z, k, q, flip=True)
    sig_z = np.cov(z_reshaped)[: q, q : 2*q]
    diag_z = np.diag(sig_z)

    # Resimulate with unitizing data
    y, x, u, z, C_new = simulate_driven_bernoulli_lds(N,x0,Q0,A,B,Q,C,D,d,muu,Qu,diag_z=diag_z,inputs=u)
    
    # moment conversion
    y_reshaped = future_past_Hankel_order_stream(y, k, q).T
    u_reshaped = future_past_Hankel_order_stream(u, k, m).T
    mu_zs, mu_us, sigma_zz, sigma_uu, sigma_zu = fit_mu_sigma_bernoulli_driven(y_reshaped, u_reshaped)

    # rearrange sigma, get estimate of covariance w 
    sigma_zz_full = tril_to_full(sigma_zz, 2 * k * q)
    sigma_what = get_sigmaw_driven(sigma_uu, sigma_zz_full, sigma_zu)

    # cholesky decompose R
    R = get_R(sigma_what)

    # run n4sid
    Ahat,Bhat,Chat,Dhat,_,_,_,ss = driven_n4sid(R,k,m,p,q)

    # Store results
    np.savez('data/em-inits/datasetD_q1/bestLDS-bigC-%d.npz' % sdx, seed=seed, A=A, B=B, C=C, C_new=C_new, D=D, 
                Q=Q, Q0=Q0, x0=x0, muu=muu, Qu=Qu, y=y, z=z, u=u, x=x, Ahat=Ahat, Bhat=Bhat, Chat=Chat, Dhat=Dhat)
    data_dict = {'seed':seed, 'A':A, 'B':B, 'C':C, 'C_new':C_new, 'D':D, 'Q':Q, 'Q0':Q0, 'x0':x0, 'muu':muu, 
                 'Qu':Qu, 'y':y, 'z':z, 'u':u, 'x':x, 'Ahat':Ahat, 'Bhat':Bhat, 'Chat':Chat, 'Dhat':Dhat}
    sio.savemat('data/em-inits/datasetD_q1/bestLDS-bigC-%d.mat' % sdx, data_dict)

0
1
2
3
4
5
6


In [4]:
np.set_printoptions(precision=3)

A_errs = np.zeros(7)
C_errs = np.zeros(7)
D_errs = np.zeros(7)
gain_errs = np.zeros(7)
for sdx in range(7):
    d = np.load('data/em-inits/datasetD_q1/best-lds-%d.npz' % sdx, allow_pickle=True)
    A = d['A']
    B = d['B']
    C = d['C_new']
    D = d['D']
    Ahat = d['Ahat']
    Bhat = d['Bhat']
    Chat = d['Chat']
    Dhat = d['Dhat']
    
    ## Check error
    # Mean absolute eigenvalue difference for A
    A_eig = np.sort(np.linalg.eig(A)[0])
    Ahat_eig = np.sort(np.linalg.eig(Ahat)[0])
    A_errs[sdx] = np.mean(np.abs(A_eig - Ahat_eig))
    
    # Subspace angle for C
    Cangle = subspace_angles(C_new, Chat)[0] # subspace angle
    C_errs[sdx] = Cangle
    
    # Elementwise error for D
    D_err = np.mean(np.abs(Dhat - D))
    D_errs[sdx] = D_err
    
    # Net error through gain matrix
    true_gain = C_new @ np.linalg.inv(np.eye(p) - A) @ B + D
    est_gain = Chat @ np.linalg.inv(np.eye(p) - Ahat) @ Bhat + Dhat
    gain_err = np.mean(np.abs(true_gain - est_gain))
    gain_errs[sdx] = gain_err
    
print('A errors:')
print('\tMean: %.3f' % np.mean(A_errs))
print('\tRaw:', A_errs)
print()

print('C errors:')
print('\tMean: %.3f' % np.mean(C_errs))
print('\tRaw:', C_errs)
print()

print('D errors:')
print('\tMean: %.3f' % np.mean(D_errs))
print('\tRaw:', D_errs)
print()

print('Gain errors:')
print('\tMean: %.3f' % np.mean(gain_errs))
print('\tRaw:', gain_errs)
print()

A errors:
	Mean: 0.543
	Raw: [0.546 0.545 0.544 0.541 0.541 0.544 0.543]

C errors:
	Mean: 0.000
	Raw: [0. 0. 0. 0. 0. 0. 0.]

D errors:
	Mean: 0.009
	Raw: [0.002 0.008 0.035 0.002 0.005 0.004 0.007]

Gain errors:
	Mean: 0.100
	Raw: [0.057 0.119 0.216 0.078 0.054 0.071 0.107]

